In [2]:
!pip install tensorflow
!pip install keras

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from keras.layers import Conv2D , Dense , Dropout , BatchNormalization , LeakyReLU , Input , Reshape , Flatten
from keras.models import Model , Sequential
from keras.datasets import mnist

In [4]:
############ DATASET PREPARATION
(x_train , y_train) , (x_test , y_test) = mnist.load_data()

x_train = x_train/255.0 * 2 -1
x_test = x_test/255.0 * 2 - 1

print(x_train.shape)
print(x_test.shape)

11490434/11490434 [==============================] - 0s 0us/step
(60000, 28, 28)
(10000, 28, 28)


In [5]:
_ , W , H = x_train.shape
T = W * H
x_train = x_train.reshape((_ , T))
x_test = x_test.reshape((10000 , T))
print(x_train.shape)

(60000, 784)


In [6]:
LATENT_DIM = 100
BATCH_SIZE = 32
EPOCH = 30000
real_labels = np.ones(BATCH_SIZE)
fake_labels = np.zeros(BATCH_SIZE)

In [7]:
def bulid_generator_model():
  i = Input(shape = (LATENT_DIM , ))
  x = Dense(256 , activation = LeakyReLU(alpha = 0.2))(i)
  x = BatchNormalization(momentum = 0.7)(x)
  x = Dense(512 , activation = LeakyReLU(alpha = 0.2))(x)
  x = BatchNormalization(momentum = 0.7)(x)
  x = Dense(1024 , activation = LeakyReLU(alpha = 0.2))(x)
  x = BatchNormalization(momentum = 0.7)(x)
  out = Dense(T , activation = 'tanh')(x)

  model = Model(i , out)

  return model

In [8]:
generator_model = bulid_generator_model()

print(generator_model.summary())

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 100)]             0         
                                                                 
 dense (Dense)               (None, 256)               25856     
                                                                 
 batch_normalization (Batch  (None, 256)               1024      
 Normalization)                                                  
                                                                 
 dense_1 (Dense)             (None, 512)               131584    
                                                                 
 batch_normalization_1 (Bat  (None, 512)               2048      
 chNormalization)                                                
                                                                 
 dense_2 (Dense)             (None, 1024)              525312

In [9]:
def build_discriminator_model():
  i = Input(shape = (T,))
  x = Flatten()(i)
  x = Dense(512 , activation = LeakyReLU(alpha = 0.2))(x)
  # x = BatchNormalization(momentum = 0.7)(x)
  x = Dense(256 , activation = LeakyReLU(alpha = 0.2))(x)
  # x = BatchNormalization(momentum = 0.7)(x)
  out = Dense(1 , activation = 'sigmoid')(x)

  model = Model(i , out)

  return model

In [10]:
discriminator_model = build_discriminator_model()
discriminator_model.compile(loss = 'binary_crossentropy' , optimizer = 'adam' , metrics = ['accuracy'])
discriminator_model.trainable = False
print(discriminator_model.summary())

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 784)]             0         
                                                                 
 flatten (Flatten)           (None, 784)               0         
                                                                 
 dense_4 (Dense)             (None, 512)               401920    
                                                                 
 dense_5 (Dense)             (None, 256)               131328    
                                                                 
 dense_6 (Dense)             (None, 1)                 257       
                                                                 
Total params: 533505 (2.04 MB)
Trainable params: 0 (0.00 Byte)
Non-trainable params: 533505 (2.04 MB)
_________________________________________________________________
None


In [11]:
z = Input(shape = (LATENT_DIM , ))
img = generator_model(z)

In [12]:
pred_label = discriminator_model(img)

In [13]:
gan_model = Model(z , pred_label)
print(gan_model.summary())

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 100)]             0         
                                                                 
 model (Functional)          (None, 784)               1493520   
                                                                 
 model_1 (Functional)        (None, 1)                 533505    
                                                                 
Total params: 2027025 (7.73 MB)
Trainable params: 1489936 (5.68 MB)
Non-trainable params: 537089 (2.05 MB)
_________________________________________________________________
None


In [14]:
gan_model.compile(loss = 'binary_crossentropy' , optimizer = 'adam')

In [15]:
d_losses = []
g_losses = []

if not os.path.exists('gan_images'):
  os.makedirs('gan_images')

In [16]:
def showImage(epoch):
  noise = np.random.randn(25 , LATENT_DIM)
  imgs = generator_model.predict(noise)
  imgs = 0.5 * imgs + 0.5
  k = 1
  n = 5
  plt.figure(figsize = (16 , 16))
  for i in range(n):
    for j in range(n):
      plt.subplot(n , n , k)
      plt.imshow(imgs[k-1].reshape(28 , 28) , cmap = 'gray')
      plt.axis('off')
      k+=1
  plt.savefig(f'{epoch+1}.png')

In [ ]:
for epoch in range(EPOCH):
  ############ TRAINING DISCRIMINATOR
  discriminator_model.trainable = True

  idx = np.random.randint(0 , x_train.shape[0] , BATCH_SIZE)
  real_images = x_train[idx].reshape((-1 , 28*28))

  noise = np.random.randn(BATCH_SIZE , LATENT_DIM)
  fake_images = generator_model.predict(noise)

  d_loss_real , d_acc_real = discriminator_model.train_on_batch(real_images , real_labels)
  d_loss_fake , d_acc_fake = discriminator_model.train_on_batch(fake_images , fake_labels)

  d_loss = 0.5 * (d_loss_real + d_loss_fake)
  d_acc = 0.5 * (d_acc_real + d_acc_fake)


  ############### TRAINING GENERATOR
  discriminator_model.trainable = False

  noise = np.random.randn(BATCH_SIZE , LATENT_DIM)
  g_loss = gan_model.train_on_batch(noise , real_labels)


  d_losses.append(d_loss)
  g_losses.append(g_loss)

  if epoch % 100 == 0 :
    print(f"epoch : {epoch+1}/{EPOCH} -- d_loss : {d_loss:.2f} -- d_acc : {d_acc:.2f} -- g_loss : {g_loss:.2f}")

  if epoch % 200 == 0:
    showImage(epoch)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')